# 2D Isostatic gravity inversion - True Model

Este [IPython Notebook](http://ipython.org/videos.html#the-ipython-notebook) utiliza a biblioteca de código aberto [Fatiando a Terra](http://fatiando.org/)

In [1]:
#%matplotlib inline
import numpy as np
from scipy.misc import derivative
import scipy as spy
from scipy import interpolate
import matplotlib
matplotlib.use('TkAgg', force=True)
import matplotlib.pyplot as plt
import math
import cPickle as pickle
import datetime
import string as st

from scipy.misc import imread

from __future__ import division

from fatiando import gravmag, mesher, utils, gridder
from fatiando.mesher import Prism, Polygon
from fatiando.gravmag import prism
from fatiando.utils import ang2vec, si2nt, contaminate
from fatiando.gridder import regular
from fatiando.vis import mpl

from numpy.testing import assert_almost_equal
from numpy.testing import assert_array_almost_equal
from pytest import raises

plt.rc('font', size=16)

/Users/macelabastos/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
import functions as fc

## Observation coordinates.

In [3]:
# Model`s limits
ymin = 0.0
ymax = 195000.0
zmin = -1000.0
zmax = 37400.0
xmin = -100000.0
xmax = 100000.0

area = [ymin, ymax, zmax, zmin]

In [4]:
ny = 150 # number of observation datas and number of prisms along the profile
# coordinates defining the horizontal boundaries of the
# adjacent columns along the profile
y = np.linspace(ymin, ymax, ny)

In [5]:
# coordinates of the center of the columns forming the
# interpretation model
n = ny - 1
dy = (ymax - ymin)/n
ycmin = ymin + 0.5*dy
ycmax = ymax - 0.5*dy
yc = np.reshape(np.linspace(ycmin, ycmax, n),(n,1))
x = np.zeros_like(yc)
z = np.zeros_like(yc)-150.0

In [6]:
## Edge extension (observation coordinates)
sigma = 2.0
edge = sigma*dy*n

## Model parameters

In [7]:
# Model densities
# Indices and polygons relationship:
# cc = continental crust layer
# oc = ocean crust layer
# w = water layer
# s = sediment layer
# m = mantle layer
dw = np.array([1030.0])
ds = np.array([2600.0])
dcc = np.array([2790.0])
doc = np.array([2850.0])
dm = np.array([3250.0])
#dc = dcc
# coordinate defining the horizontal boundaries of the continent-ocean boundary
COT = 117000.0

# list defining crust density variance
dc = np.zeros_like(yc)
aux = yc <= COT
for i in range(len(yc[aux])):
    dc[i] = dcc
for i in range(len(yc[aux]),n):
    dc[i] = doc
# defining sediments layers density vector
ds = np.reshape(np.repeat(ds,n),(n,1))
    
# S0 => isostatic compensation surface (Airy's model)
# SR = S0+dS0 => reference Moho (Forward modeling)
S0 = np.array([29500.0])
dS0 = np.array([1200.0])

## For display

## Water bottom

In [8]:
bath_picks = np.array([[  3.93145161e+02,   1.36801334e+02],
       [  2.75201613e+03,   4.85343517e+02],
       [  6.68346774e+03,   1.00815679e+03],
       [  9.82862903e+03,   1.18242788e+03],
       [  1.94606855e+04,   1.44383452e+03],
       [  3.38104839e+04,   1.79237671e+03],
       [  5.16985887e+04,   1.96664780e+03],
       [  7.07661290e+04,   2.05378334e+03],
       [  8.86542339e+04,   2.14091889e+03],
       [  9.39616935e+04,   2.22805444e+03],
       [  1.14012097e+05,   2.48946107e+03],
       [  1.26003024e+05,   2.40232553e+03],
       [  1.34259073e+05,   2.31518998e+03],
       [  1.37207661e+05,   2.48946107e+03],
       [  1.42711694e+05,   2.66373217e+03],
       [  1.65514113e+05,   2.83800326e+03],
       [  1.94606855e+05,   2.83800326e+03]])

In [9]:
# change the coordinates of the extremum points in order to
# avoid problems for constructing the interpolator
bath_picks[0,0] = ymin
bath_picks[-1,0] = ymax

In [10]:
bathymetry = fc.surface_interpolate_function(bath_picks,yc)

In [11]:
#np.savetxt('../data/C-model-rifted-margin-bathymetry.txt', bathymetry, fmt='%.18f')

In [12]:
# layer water thickness
tw = np.reshape(bathymetry,(n,1))

# Sediments sufaces

In [13]:
sed_picks = np.array([[    589.71774194,    1863.8278536 ],
       [   2162.2983871 ,    2125.23449132],
       [   4914.31451613,    2125.23449132],
       [   5897.17741935,    3083.72549628],
       [  10811.49193548,    3170.86104218],
       [  12384.07258065,    2822.31885856],
       [  25357.86290323,    4565.02977667],
       [  28503.02419355,    5610.65632754],
       [  31451.61290323,    5697.79187345],
       [  35383.06451613,    7353.36724566],
       [  38331.65322581,    7701.90942928],
       [  40887.09677419,    8311.85825062],
       [  44032.25806452,    7091.96060794],
       [  52484.87903226,    8573.26488834],
       [  53467.74193548,    9706.02698511],
       [  58382.05645161,   11448.73790323],
       [  62510.08064516,   10751.65353598],
       [  70569.55645161,   12842.90663772],
       [  72731.85483871,   12494.36445409],
       [  74894.15322581,   11187.33126551],
       [  78039.31451613,   10925.92462779],
       [  82953.62903226,   12320.09336228],
       [  86885.08064516,   11535.87344913],
       [  91406.25      ,   10664.51799007],
       [ 100645.16129032,   10490.24689826],
       [ 108704.63709677,    9880.29807692],
       [ 115388.10483871,   10664.51799007],
       [ 118533.26612903,    8747.53598015],
       [ 194213.70967742,    8747.53598015]])

In [14]:
# change the coordinates of the extremum points in order to
# avoid problems for constructing the interpolator
sed_picks[0,0] = ymin
sed_picks[-1,0] = ymax

In [15]:
basement = fc.surface_interpolate_function(sed_picks,yc)
for i in range(len(basement)):
    if basement[i] < bathymetry[i]:
        basement[i] = bathymetry[i]

In [16]:
# layer sediments thickness
ts = basement - tw

In [17]:
#np.savetxt('../data/C-model-rifted-margin-true-basement-surface.txt', basement, fmt='%.18f')

## Moho surface

In [18]:
moho_picks = np.array([[    589.71774194,   26016.05846774],
       [   3734.87903226,   25406.1096464 ],
       [   8649.19354839,   24796.16082506],
       [  13170.36290323,   24186.21200372],
       [  18870.96774194,   23489.12763648],
       [  25161.29032258,   22704.90772333],
       [  29485.88709677,   21920.68781017],
       [  35383.06451613,   20700.79016749],
       [  41869.95967742,   19045.21479529],
       [  45015.12096774,   17825.31715261],
       [  47767.13709677,   16344.01287221],
       [  49929.43548387,   15385.52186725],
       [  52091.73387097,   14775.57304591],
       [  54843.75      ,   14252.75977047],
       [  57792.33870968,   14252.75977047],
       [  60740.92741935,   14427.03086228],
       [  64082.66129032,   14688.4375    ],
       [  66638.10483871,   14949.84413772],
       [  70766.12903226,   14862.70859181],
       [  74501.00806452,   14688.4375    ],
       [  76663.30645161,   14775.57304591],
       [  79218.75      ,   15124.11522953],
       [  81381.0483871 ,   16082.60623449],
       [  83936.49193548,   16692.55505583],
       [  86885.08064516,   17215.36833127],
       [  91602.82258065,   17651.04606079],
       [  97500.        ,   17476.77496898],
       [ 106542.33870968,   17302.50387717],
       [ 115388.10483871,   17215.36833127],
       [ 128951.61290323,   17128.23278536],
       [ 137207.66129032,   16692.55505583],
       [ 147036.29032258,   16344.01287221],
       [ 150574.59677419,   16344.01287221],
       #[ 155685.48387097,   15995.47068859],
       #[ 160010.08064516,   15559.79295906],
       #[ 166890.12096774,   15211.25077543],
       #[ 172590.72580645,   15211.25077543],
       #[ 178881.0483871 ,   15385.52186725],
       #[ 185957.66129032,   15211.25077543],
       #[ 191658.26612903,   15124.11522953],
       #[ 194213.70967742,   14862.70859181]])
        [ 194213.70967742,   16344.01287221]])

In [19]:
# change the coordinates of the extremum points in order to
# avoid problems for constructing the interpolator
moho_picks[0,0] = ymin
moho_picks[-1,0] = ymax

In [20]:
moho = fc.surface_interpolate_function(moho_picks,yc)
for i in range(len(moho)):
    if moho[i] < basement[i]:
        moho[i] = basement[i]

In [21]:
# layer mantle thickness
tm = S0 - moho

# layer crust thickness
toc = moho - tw - ts

In [22]:
#np.savetxt('../data/C-model-rifted-margin-true-moho-surface.txt', moho, fmt='%.18f')

## Known depths

In [23]:
print '[', np.reshape(yc,(n,))[13], ',' , np.reshape(basement,(n,))[13], '],'
print '[', np.reshape(yc,(n,))[35], ',' , np.reshape(basement,(n,))[35], '],'
print '[', np.reshape(yc,(n,))[70], ',' , np.reshape(basement,(n,))[70], '],'
print '[', np.reshape(yc,(n,))[95], ',' , np.reshape(basement,(n,))[95], '],'
print '[', np.reshape(yc,(n,))[125], ',' , np.reshape(basement,(n,))[125], ']'

[ 17667.7852349 , 3532.05622149 ],
[ 46459.7315436 , 7517.37029671 ],
[ 92265.1006711 , 10648.3177215 ],
[ 124983.221477 , 8747.53598015 ],
[ 164244.966443 , 8747.53598015 ]


In [30]:
# Basement surface known values
base_known = np.array([[ 17667.7852349 , 3532.05622149 ],
[ 46459.7315436 , 7517.37029671 ],
#[ 92265.1006711 , 10648.3177215 ],
#[ 124983.221477 , 8747.53598015 ],
[ 164244.966443 , 8747.53598015 ]
                      ])

np.savetxt('../data/C-model-rifted-margin-basement-known-depths.txt', base_known, fmt='%.18f')

In [25]:
print '[', np.reshape(yc,(n,))[13], ',' , np.reshape(moho,(n,))[13], '],'
print '[', np.reshape(yc,(n,))[35], ',' , np.reshape(moho,(n,))[35], '],'
print '[', np.reshape(yc,(n,))[125], ',' , np.reshape(moho,(n,))[125], ']'

[ 17667.7852349 , 23636.2558343 ],
[ 46459.7315436 , 17047.738923 ],
[ 164244.966443 , 16344.0128722 ]


In [28]:
# Moho surface known values
moho_known = np.array([#[ 17667.7852349 , 23636.2558343 ],
[ 46459.7315436 , 17047.738923 ]#,
#[ 164244.966443 , 16344.0128722 ]
                      ])

np.savetxt('../data/C-model-rifted-margin-moho-known-depths.txt', moho_known, fmt='%.18f')

## Synthetic data

In [ ]:
# parameters vector
psyn = []
psyn = np.vstack((ts, tm, dS0))

In [ ]:
# prisms calculation by Fatiando a Terra
prism_w_syn = fc.prism_w_function(xmax,xmin,dy,edge,dw,dcc,tw,yc)
prism_s_syn = fc.prism_s_function(xmax,xmin,dy,edge,ds,dcc,tw,psyn,yc)
prism_c_syn = fc.prism_c_function(xmax,xmin,dy,edge,S0,dcc,dc,tw,psyn,yc)
prism_m_syn = fc.prism_m_function(xmax,xmin,dy,edge,S0,dcc,dm,psyn,yc)

In [ ]:
# z component of gravity calculation by Fatiando a Terra
gzw_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_w_syn)
gzs_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_s_syn[0])
gzc_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_c_syn)
gzm_syn = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_m_syn)

#Observed data calculation:
gsyn = gzw_syn + gzs_syn + gzc_syn + gzm_syn
#gsyn = fc.g_function(x,yc,z,gzw_syn,prism_s_syn,prism_c_syn,prism_m_syn)

In [ ]:
#np.savetxt('../data/C-model-rifted-margin-synthetic-gravity-data.txt', gsyn, fmt='%.18f')

In [ ]:
sgm = 9.81*(10**(-6))*(dw*tw + ds*ts + dc*(S0-tw-ts-tm)+dm*tm)

## Model plot

In [ ]:
polygons_water = []
for (yi, twi) in zip(yc, bathymetry):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_water.append(Polygon(np.array([[y1, y2, y2, y1], 
                                            [0.0, 0.0, twi, twi]]).T,
                                               props={'density': dw - dcc}))
polygons_sediments = []
for (yi, twi, si, dsi) in zip(yc, np.reshape(bathymetry,(n,)), np.reshape(basement,(n,)), ds):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_sediments.append(Polygon(np.array([[y1, y2, y2, y1], 
                                                [twi, twi, si, si]]).T,
                                                    props={'density': ds - dcc}))

polygons_crust = []
for (yi, si, Si, dci) in zip(yc, np.reshape(basement,(n,)), np.reshape(moho,(n,)), dc):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_crust.append(Polygon(np.array([[y1, y2, y2, y1], 
                                            [si, si, Si, Si]]).T,
                                                props={'density': dci - dcc})) 

polygons_mantle = []
for (yi, Si) in zip(yc, np.reshape(moho,(n,))):
    y1 = yi - 0.5*dy
    y2 = yi + 0.5*dy

    polygons_mantle.append(Polygon(np.array([[y1, y2, y2, y1], 
                                                [Si, Si, S0+dS0, S0+dS0]]).T,
                                                    props={'density': dm - dcc}))

In [ ]:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(12,16))

import matplotlib.gridspec as gridspec
heights = [8, 8, 8, 1]
gs = gridspec.GridSpec(4, 1, height_ratios=heights)
ax3 = plt.subplot(gs[0])
ax6 = plt.subplot(gs[1])
ax4 = plt.subplot(gs[2])
ax5 = plt.subplot(gs[3])

ax3.axhline(y=0.0, xmin=ymin, xmax=ymax, color='k', linestyle='--', linewidth=1)
ax3.plot(0.001*yc, gsyn, '-g', linewidth=2, label='simulated data')
ax3.set_xlim(0.001*ymin, 0.001*ymax)
ax3.set_ylabel('gravity disturbance (mGal)', fontsize=16)
ax3.set_xticklabels(['%g'% (l) for l in ax3.get_xticks()], fontsize=14)
ax3.set_yticklabels(['%g'% (l) for l in ax3.get_yticks()], fontsize=14)
ax3.legend(loc='best', fontsize=14, facecolor='silver')

ax6.plot(0.001*yc, sgm, '-g', linewidth=2, label='lithostatic stress')
ax6.set_xlim(0.001*ymin, 0.001*ymax)
ax6.set_ylim(770,860)
ax6.set_ylabel('Lithostatic Stress (MPa)', fontsize=16)
ax6.set_xticklabels(['%g'% (l) for l in ax6.get_xticks()], fontsize=14)
ax6.set_yticklabels(['%g'% (l) for l in ax6.get_yticks()], fontsize=14)
ax6.legend(loc='best', fontsize=14, facecolor='silver')

ax4.axhline(y=0.0, xmin=ymin, xmax=ymax, color='k', linestyle='-', linewidth=1)
aux = yc <= COT
for (pwi) in (polygons_water):
    tmpx = [x for x in pwi.x]
    tmpx.append(pwi.x[0])
    tmpy = [y for y in pwi.y]
    tmpy.append(pwi.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='lightskyblue')
for (psi) in (polygons_sediments):
    tmpx = [x for x in psi.x]
    tmpx.append(psi.x[0])
    tmpy = [y for y in psi.y]
    tmpy.append(psi.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='tan')
for (pci) in (polygons_crust[:len(yc[aux])]):
    tmpx = [x for x in pci.x]
    tmpx.append(pci.x[0])
    tmpy = [y for y in pci.y]
    tmpy.append(pci.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='orange')
for (pcoi) in (polygons_crust[len(yc[aux]):n]):
    tmpx = [x for x in pcoi.x]
    tmpx.append(pcoi.x[0])
    tmpy = [y for y in pcoi.y]
    tmpy.append(pcoi.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='olive')
for (pmi) in (polygons_mantle):
    tmpx = [x for x in pmi.x]
    tmpx.append(pmi.x[0])
    tmpy = [y for y in pmi.y]
    tmpy.append(pmi.y[0])
    ax4.plot(tmpx, tmpy, linestyle='None')
    ax4.fill(tmpx, tmpy, color='pink')

#ax4.axhline(y=S0, xmin=ymin, xmax=ymax, color='w', linestyle='--', linewidth=3)
#ax4.axvline(x=COT, ymin=ybottom, ymax=ytop, linestyle=':', color='w', linewidth=3, label='COT')
ax4.plot(yc, tw, '-k', linewidth=3)
ax4.plot(yc, basement, '-k', linewidth=3, label='true surfaces')
ax4.plot(yc, moho, '-k', linewidth=3)
ax4.axhline(y=S0+dS0, xmin=ymin, xmax=ymax, color='k', linestyle='-', linewidth=3)
#ax4.set_ylim((S0+dS0), zmin)
ax4.set_ylim((39000.0), zmin)
ax4.set_xlim(ymin, ymax)
ax4.set_xlabel('y (km)', fontsize=16)
ax4.set_ylabel('z (km)', fontsize=16)
ax4.set_xticklabels(['%g'% (0.001*l) for l in ax4.get_xticks()], fontsize=14)
ax4.set_yticklabels(['%g'% (0.001*l) for l in ax4.get_yticks()], fontsize=14)
#ax4.legend(loc='lower right', fontsize=14, facecolor='silver')    


X, Y = fig.get_dpi()*fig.get_size_inches()
plt.title('Density (kg/m$^{3}$)', fontsize=18)
ax5.axis('off')

layers_list1 = ['water', 'sediment', 'continental', 'oceanic', 'mantle']
layers_list2 = ['', '', 'crust', 'crust', '']
colors_list = ['lightskyblue', 'tan', 'orange', 'olive', 'pink']
density_list = ['1030', '2600', '2790', '2800', '3200']
ncols = len(colors_list)
nrows = 1
h = Y / nrows
w = X / (ncols + 1)

i=ncols-1
for color, density, layers1, layers2 in zip(colors_list, density_list, layers_list1, layers_list2):
    col = i // nrows
    row = i % nrows
    x = X - (col*w) - w
    
    yi_line = Y
    yf_line = Y - Y*0.15
    yi_text1 = Y - Y*0.2
    yi_text2 = Y - Y*0.27
    yi_text3 = Y - Y*0.08
    
    i-=1
    poly = Polygon(np.array([[x, x+w*0.75, x+w*0.75, x], [yi_line, yi_line, yf_line, yf_line]]).T)
    tmpx = [x for x in poly.x]
    tmpx.append(poly.x[0])
    tmpy = [y for y in poly.y]
    tmpy.append(poly.y[0])
    ax5.plot(tmpx, tmpy, linestyle='-', color='k', linewidth=1)
    ax5.fill(tmpx, tmpy, color=color)
    ax5.text(x+w*0.375, yi_text1, layers1, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='top')
    ax5.text(x+w*0.375, yi_text2, layers2, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='top')
    ax5.text(x+w*0.375, yi_text3, density, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='center')

plt.tight_layout()    
    
#mpl.savefig('../manuscript/figures/C-model-rifted-margin-true-model-grafics.png', dpi='figure', bbox_inches='tight')
plt.show()      

In [ ]:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(12,7))

import matplotlib.gridspec as gridspec
heights = [8, 1]
gs = gridspec.GridSpec(2, 1, height_ratios=heights)
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

ax1.axhline(y=0.0, xmin=ymin, xmax=ymax, color='k', linestyle='-', linewidth=1)
aux = yc <= COT
for (pwi) in (polygons_water):
    tmpx = [x for x in pwi.x]
    tmpx.append(pwi.x[0])
    tmpy = [y for y in pwi.y]
    tmpy.append(pwi.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='lightskyblue')
for (psi) in (polygons_sediments):
    tmpx = [x for x in psi.x]
    tmpx.append(psi.x[0])
    tmpy = [y for y in psi.y]
    tmpy.append(psi.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='tan')
for (pci) in (polygons_crust[:len(yc[aux])]):
    tmpx = [x for x in pci.x]
    tmpx.append(pci.x[0])
    tmpy = [y for y in pci.y]
    tmpy.append(pci.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='orange')
for (pcoi) in (polygons_crust[len(yc[aux]):n]):
    tmpx = [x for x in pcoi.x]
    tmpx.append(pcoi.x[0])
    tmpy = [y for y in pcoi.y]
    tmpy.append(pcoi.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='olive')
for (pmi) in (polygons_mantle):
    tmpx = [x for x in pmi.x]
    tmpx.append(pmi.x[0])
    tmpy = [y for y in pmi.y]
    tmpy.append(pmi.y[0])
    ax1.plot(tmpx, tmpy, linestyle='None')
    ax1.fill(tmpx, tmpy, color='pink')

#ax1.axhline(y=S0, xmin=ymin, xmax=ymax, color='w', linestyle='--', linewidth=3)
#ax1.axvline(x=COT, ymin=ybottom, ymax=ytop, linestyle=':', color='w', linewidth=3, label='COT')
ax1.plot(yc, tw, '-k', linewidth=3)
ax1.plot(yc, basement, '-k', linewidth=3, label='true surfaces')
ax1.plot(yc, moho, '-k', linewidth=3)
ax1.axhline(y=S0+dS0, xmin=ymin, xmax=ymax, color='k', linestyle='-', linewidth=3)
#ax1.set_ylim((S0+dS0), zmin)
ax1.set_ylim((39000.0), zmin)
ax1.set_xlim(ymin, ymax)
ax1.set_xlabel('y (km)', fontsize=16)
ax1.set_ylabel('z (km)', fontsize=16)
ax1.set_xticklabels(['%g'% (0.001*l) for l in ax1.get_xticks()], fontsize=14)
ax1.set_yticklabels(['%g'% (0.001*l) for l in ax1.get_yticks()], fontsize=14)
#ax1.legend(loc='lower right', fontsize=14, facecolor='silver')    


X, Y = fig.get_dpi()*fig.get_size_inches()
plt.title('Density (kg/m$^{3}$)', fontsize=18)
ax2.axis('off')

layers_list1 = ['water', 'sediment', 'continental', 'oceanic', 'mantle']
layers_list2 = ['', '', 'crust', 'crust', '']
colors_list = ['lightskyblue', 'tan', 'orange', 'olive', 'pink']
density_list = ['1030', '2600', '2790', '2800', '3200']
ncols = len(colors_list)
nrows = 1
h = Y / nrows
w = X / (ncols + 1)

i=ncols-1
for color, density, layers1, layers2 in zip(colors_list, density_list, layers_list1, layers_list2):
    col = i // nrows
    row = i % nrows
    x = X - (col*w) - w
    
    yi_line = Y
    yf_line = Y - Y*0.15
    yi_text1 = Y - Y*0.2
    yi_text2 = Y - Y*0.27
    yi_text3 = Y - Y*0.08
    
    i-=1
    poly = Polygon(np.array([[x, x+w*0.75, x+w*0.75, x], [yi_line, yi_line, yf_line, yf_line]]).T)
    tmpx = [x for x in poly.x]
    tmpx.append(poly.x[0])
    tmpy = [y for y in poly.y]
    tmpy.append(poly.y[0])
    ax2.plot(tmpx, tmpy, linestyle='-', color='k', linewidth=1)
    ax2.fill(tmpx, tmpy, color=color)
    ax2.text(x+w*0.375, yi_text1, layers1, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='top')
    ax2.text(x+w*0.375, yi_text2, layers2, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='top')
    ax2.text(x+w*0.375, yi_text3, density, fontsize=(w*0.14), horizontalalignment='center', verticalalignment='center')

plt.tight_layout()    
    
#mpl.savefig('../manuscript/figures/C-model-rifted-margin-true-model.png', dpi='figure', bbox_inches='tight')
plt.show()      